In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='7um'

In [4]:
design.get_chip_z('Q_chip')

'7um'

In [5]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="110um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [7]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [9]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [11]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [12]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [13]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [33]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [34]:
hfss.start()

INFO 03:36PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:36PM [load_ansys_project]: 	Opened Ansys App
INFO 03:36PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 03:36PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project1
INFO 03:36PM [connect_design]: No active design found (or error getting active design).
INFO 03:36PM [connect]: 	 Connected to project "Project1". No design detected


True

In [35]:
hfss.activate_ansys_design("flip7um_V6", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

03:36PM 33s WARNING [activate_ansys_design]: The design_name=flip7um_V6 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 03:36PM [connect_design]: 	Opened active design
	Design:    flip7um_V6 [Solution type: Eigenmode]
WARNING 03:36PM [connect_setup]: 	No design setup detected.
WARNING 03:36PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:36PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [36]:
hfss.close()


True

# LOM Analysis

In [14]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [15]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.88',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.88',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [19]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:31PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpdszymu64.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 1, False
INFO 04:31PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpa3vbz755.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 1, False
INFO 04:31PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmptz5c1e7n.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 2, False
INFO 04:31PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp638wtjt2.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 3, False
INFO 04:31PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp8repp5cu.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5880000000, Maxwell, 4, False
INFO 04:31

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,129.250000,-12.155,-0.069086,-115.810
pad_top_Q0,-12.155000,129.090,-16.502000,-99.131
quad_coupler_c0,-0.069086,-16.502,92.357000,-75.740
SignalNet1,-115.810000,-99.131,-75.740000,356.130


In [20]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 7.20
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.437819 [GHz]
EC 304.592113 [MHz]
EJ 13.616300 [GHz]
alpha -353.456585 [MHz]
dispersion 30.615761 [KHz]
Lq 11.995161 [nH]
Cq 63.593991 [fF]
T1 156.470811 [us]

**Coupling Properties**

tCqbus1 -4.883993 [fF]
gbus1_in_MHz -76.965304 [MHz]
χ_bus1 -1.150192 [MHz]
1/T1bus1 1017.154201 [Hz]
T1bus1 156.470811 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.380592,429.724838,13.6163,-518.482661,537.163829,[-84.051783649056],[-6.723767372843621],6.723767,84.051784
2,5.681313,334.52799,13.6163,-391.864243,70.409621,[-53.27329686319154],[-0.7804058641043077],0.780406,53.273297
3,5.539377,316.881696,13.6163,-369.145544,43.725697,[-76.38725890458001],[-1.3053984954876754],1.305398,76.387259
4,5.472573,308.766284,13.6163,-358.773172,34.639704,[-81.08675951208397],[-1.339398063897869],1.339398,81.086760
5,5.457215,306.917619,13.6163,-356.417008,32.806522,[-81.02724277827055],[-1.3093001358113507],1.309300,81.027243
6,5.443232,305.240126,13.6163,-354.281131,31.213609,[-77.35172755643711],[-1.170442199490946],1.170442,77.351728
7,5.437819,304.592113,13.6163,-353.456585,30.615761,[-76.96530434784809],[-1.150191919288642],1.150192,76.965304


In [21]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
